In [1]:
# Define file path and paramaters
agilysis_file_path = "/Users/hucongying/NCN/Quietway_Assessment/Sustran_data/Sustran.gpkg"
ncn_file_path = "/Users/hucongying/NCN/Quietway_Assessment/National_Cycle_Network_(Public).geojson"
settlement_path = '/Users/hucongying/NCN/Quietway_Assessment/Useless Settlement Estimated 2020-shapefiles/Settlements_2020_MHW.shp'
buffer=10

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

/var/folders/74/ch5llxvd3klg6n6_l6_s335w0000gn/T/ipykernel_70864/3651934155.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopanda

In [3]:
# Read Agilysis data
agilysis = gpd.read_file(agilysis_file_path)
print(agilysis.head())

             OS Link ID    Road Classification Road Classification Number   
0  osgb4000000004292146  Classified Unnumbered                        NaN  \
1  osgb4000000004292268           Unclassified                        NaN   
2  osgb4000000004292271                 A Road                        A84   
3  osgb4000000004292724  Classified Unnumbered                        NaN   
4  osgb4000000004292780                Unknown                        NaN   

          Form of Way  Length (km) Trunk Road Highway Authority Code   
0  Single Carriageway     2.209182      false              S12000030  \
1  Single Carriageway     1.308137      false              S12000030   
2  Single Carriageway     2.058606       true              S12000030   
3  Single Carriageway     0.656623      false              S12000030   
4  Single Carriageway     0.614588      false              S12000030   

  Highway Authority Name  Speed Limit (mph) Speed Limit Evidence date  ...   
0               Stirling  

In [4]:
# Coordinate reference system 
print(agilysis.crs)  

EPSG:27700


In [5]:
# Read the ncn data

ncn = gpd.read_file(ncn_file_path)

# Preview the data
print(ncn.head())
print(ncn.crs)  # Coordinate reference system


   FID   Desc_ Greenway RouteType  RouteNo  LinkNo    RouteCat OpenStatus   
0    1  OnRoad       No       NCN    601.0     0.0  Main Route       Open  \
1    2  OnRoad       No       NCN      1.0     0.0  Main Route       Open   
2    3  OnRoad       No       NCN      1.0     0.0  Main Route       Open   
3    4  OnRoad       No       RCN     61.0     0.0         N/A       Open   
4    5  OnRoad       No       NCN      1.0     0.0  Main Route       Open   

   Surface     Quality Lighting     RoadClass   
0  Asphalt       Rough   NotLit    Minor Road  \
1  Asphalt    Standard  PartLit  Local Street   
2  Asphalt    Standard  PartLit    Minor Road   
3  Asphalt  Acceptable   NotLit    Minor Road   
4  Asphalt    Standard   NotLit    Minor Road   

                               GlobalID  SegmentID   
0  59ef8425-72cd-4787-a8cd-4cdf71d69570        828  \
1  89cd191e-dcc8-41d0-ac34-bd03caccb67d      11459   
2  1fd2cc0b-466f-4d06-abd2-b9baadb6c2ff      31285   
3  cf27eabd-8973-4bab-9a40

In [6]:
# Transfer the coordinate reference system
ncn=ncn.to_crs(epsg=27700) 

In [7]:
# View columns names
ncn.columns

Index(['FID', 'Desc_', 'Greenway', 'RouteType', 'RouteNo', 'LinkNo',
       'RouteCat', 'OpenStatus', 'Surface', 'Quality', 'Lighting', 'RoadClass',
       'GlobalID', 'SegmentID', 'geometry'],
      dtype='object')

In [8]:
# Get dataset when routes are on road
ncn_onroad = ncn[ncn['Desc_'] == 'OnRoad']

In [9]:
print(f"Number of routes on road is {len(ncn_onroad)}")

Number of routes on road is 16678


In [10]:
# Create 10m buffer for on road ncn dataset
ncn_buffered = ncn_onroad.copy()
ncn_buffered['geometry'] = ncn_buffered.geometry.buffer(buffer)


/Users/hucongying/mambaforge/lib/python3.10/site-packages/pygeos/constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [11]:
# Merge buffers
buffer_union = ncn_buffered.unary_union

/Users/hucongying/mambaforge/lib/python3.10/site-packages/pygeos/set_operations.py:388: RuntimeWarning: invalid value encountered in unary_union
  result = lib.unary_union(collections, **kwargs)


In [13]:
import time 
# to oversee the progress
start = time.time()
agilysis['geometry_intersection'] = agilysis.geometry.intersection(buffer_union)
print("Done in", round(time.time() - start, 2), "seconds")

Done in 1158.01 seconds


/Users/hucongying/mambaforge/lib/python3.10/site-packages/pygeos/set_operations.py:129: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


In [14]:
# # Compute intersection geometry and lengths
# agilysis['geometry_intersection'] = agilysis.geometry.intersection(buffer_union)

In [15]:
# Calculate original and intersected lengths
agilysis['length_total'] = agilysis.geometry.length
agilysis['length_in_buffer'] = agilysis['geometry_intersection'].length

In [16]:
# Calculate percentage and filter
agilysis['percent_in_buffer'] = agilysis['length_in_buffer'] / agilysis['length_total']
agilysis_selected = agilysis[agilysis['percent_in_buffer'] >= 0.9]


In [17]:
agilysis_selected.head()

,OS Link ID,Road Classification,Road Classification Number,Form of Way,Length (km),Trunk Road,Highway Authority Code,Highway Authority Name,Speed Limit (mph),Speed Limit Evidence date,...,Number over Limit,Number over ACPO,Number over 15 Above,roadwidth_average,roadwidth_minimum,geometry,geometry_intersection,length_total,length_in_buffer,percent_in_buffer
0,osgb4000000004292146,Classified Unnumbered,NaN,Single Carriageway,2.209182,false,S12000030,Stirling,60,NaN,...,NaN,NaN,NaN,5.2,NaN,"MULTILINESTRING ((264275.866 734983.274, 26433...","MULTILINESTRING ((264275.866 734983.274, 26433...",2209.182012,2209.182012,1.0
3,osgb4000000004292724,Classified Unnumbered,NaN,Single Carriageway,0.656623,false,S12000030,Stirling,60,NaN,...,NaN,NaN,NaN,4.0,3.2,"MULTILINESTRING ((261274.037 707315.032, 26127...","MULTILINESTRING ((261274.037 707315.032, 26127...",656.622934,656.622934,1.0
4,osgb4000000004292780,Unknown,NaN,Single Carriageway,0.614588,false,S12000030,Stirling,30,NaN,...,NaN,NaN,NaN,5.0,4.1,"MULTILINESTRING ((255564.000 716216.000, 25558...","MULTILINESTRING ((255564.000 716216.000, 25558...",614.587727,614.587727,1.0
5,osgb4000000004293117,Classified Unnumbered,NaN,Single Carriageway,1.174929,false,S12000030,Stirling,60,NaN,...,NaN,NaN,NaN,4.4,3.1,"MULTILINESTRING ((258317.000 732240.000, 25829...","MULTILINESTRING ((258317.000 732240.000, 25829...",1174.929448,1174.929448,1.0
6,osgb4000000004293123,Classified Unnumbered,NaN,Single Carriageway,1.204896,false,S12000030,Stirling,60,NaN,...,NaN,NaN,NaN,5.8,4.0,"MULTILINESTRING ((259323.217 733055.412, 25933...","MULTILINESTRING ((259323.217 733055.412, 25933...",1204.895873,1204.895873,1.0


In [18]:
print(f"Number of Agilysis routes is {len(agilysis)}, among them {len(agilysis_selected)} have more than 90% of length in NCN 10m buffer.")


Number of Agilysis routes is 21841, among them 10478 have more than 90% of length in NCN 10m buffer.


## 10,478 routes have more than 90% of length within NCN 10m buffer among 21,841 routes.

_While Des' result is 10,819 routes. 341 gap_

In [19]:
# Export the dataset preventing rerun it again which is time costing
agilysis_selected_clean = agilysis_selected.drop(columns=['geometry_intersection'])


agilysis_selected_clean.to_file("agilysis_90pct_within_buffer.gpkg", driver="GPKG", layer="agilysis_lines")


# Assessment

### Further understand the dateset
1. If the speed data is fully modelled (i.e. no ‘field survey’ for any of the 6 key time intervals).  Condition: Count_of_not_modelled = 0 
2. If the speed data contains no 85th percentile speed for the 6 key time intervals.  Condition: Count_of_85th_speed = 0 
3. If the speed data contains no 85th percentile all day speed.  Condition: IsEmpty(F85th__ile_Speed___All_Day) 
4. If there is no volume data. Condition: IsEmpty(Modelled_AADT_Band) 

In [144]:
# Read a theh file we just exported obove
quietway_assessment = gpd.read_file("agilysis_90pct_within_buffer.gpkg", layer="agilysis_lines")

Check if routes' survey condition with values in 'Field Survey':  
'averagespeed_mf7to9capturemethod',
 'averagespeed_mf9to12capturemethod',
 'averagespeed_mf12to14capturemethod',
 'averagespeed_mf14to16capturemethod',
 'averagespeed_mf16to19capturemethod',
 'averagespeed_mf19to22capturemethod',

In [145]:
# Define the 6 fields to check
fields_to_check = [
    'averagespeed_mf7to9capturemethod',
    'averagespeed_mf9to12capturemethod',
    'averagespeed_mf12to14capturemethod',
    'averagespeed_mf14to16capturemethod',
    'averagespeed_mf16to19capturemethod',
    'averagespeed_mf19to22capturemethod',
]

# Create the new field with conditional logic
def classify_survey(row):
    count = (row[fields_to_check] == 'Field Survey').sum()
    if count == 0:
        return None
    elif count < 6:
        return 'partially surveyed'
    else:
        return 'fully surveyed'

quietway_assessment['survey_flag'] = quietway_assessment.apply(classify_survey, axis=1)


In [146]:
quietway_assessment['survey_flag']

0        partially surveyed
1        partially surveyed
2        partially surveyed
3        partially surveyed
4        partially surveyed
                ...        
10473                  None
10474                  None
10475                  None
10476                  None
10477                  None
Name: survey_flag, Length: 10478, dtype: object

### Check if 85th %ile is null, when it is not fully surveyed

In [147]:
quietway_assessment['survey_flag'].value_counts(dropna=False)


survey_flag
fully surveyed        7290
partially surveyed    2744
None                   444
Name: count, dtype: int64

In [148]:
# To explore the relationship between fields
# null_survey = quietway_assessment[quietway_assessment['survey_flag'].isnull()]
# len(null_survey)

# null_survey['85th %ile Speed Mon to Fri - 09:00 to 12:00'].isnull().sum()

# partial_survey = quietway_assessment[quietway_assessment['survey_flag'] == 'partially surveyed']
# partial_survey['85th %ile Speed Mon to Fri - 09:00 to 12:00'].isnull().sum()

# Presumably, when it is partially surveyed, there will be 85th %ile for some fields when the average speed was surveyed.

# quietway_assessment['averagespeed_mf9to12capturemethod'].isnull().sum()

# quietway_assessment['85th %ile Speed Mon to Fri - 09:00 to 12:00'].isnull().sum()

# null_survey['Modelled AADT'].isnull().sum()

# quietway_assessment['Modelled AADT B'].isnull().sum()

In [149]:
# Read the settlement data
settlement = gpd.read_file(settlement_path)

In [150]:
# Check the crs
settlement.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

## Start assessment    
1. Get rid of routes when **'85th %ile Speed - All Day', 'Modelled AADT'** are null.

In [151]:
# Filter out rows with nulls in either of the two key fields for allocating quiet values
quietway_assessment_intersect = quietway_assessment.dropna(subset=['85th %ile Speed - All Day', 'Modelled AADT'])


In [152]:
#Flag whether the feature intersects with settlement boundaries
quietway_assessment_intersect = quietway_assessment_intersect.copy()
quietway_assessment_intersect['within_settlement'] = quietway_assessment_intersect.geometry.intersects(settlement.unary_union)



/Users/hucongying/mambaforge/lib/python3.10/site-packages/pygeos/predicates.py:764: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


In [153]:
len(quietway_assessment_intersect)

9807

In [154]:
quietway_assessment_intersect['within_settlement'].sum()

6051

## 6,051 routes are intersected with settlements 
among 9,807 routes have values in both **'85th %ile Speed - All Day', 'Modelled AADT'**

_While Des' result is 6,759 intersected with settlements._ 708 gap

In [155]:
quietway_assessment_intersect.head()

,OS Link ID,Road Classification,Road Classification Number,Form of Way,Length (km),Trunk Road,Highway Authority Code,Highway Authority Name,Speed Limit (mph),Speed Limit Evidence date,...,Number over ACPO,Number over 15 Above,roadwidth_average,roadwidth_minimum,length_total,length_in_buffer,percent_in_buffer,geometry,survey_flag,within_settlement
0,osgb4000000004292146,Classified Unnumbered,NaN,Single Carriageway,2.209182,false,S12000030,Stirling,60,NaN,...,NaN,NaN,5.2,NaN,2209.182012,2209.182012,1.0,"MULTILINESTRING ((264275.866 734983.274, 26433...",partially surveyed,False
1,osgb4000000004292724,Classified Unnumbered,NaN,Single Carriageway,0.656623,false,S12000030,Stirling,60,NaN,...,NaN,NaN,4.0,3.2,656.622934,656.622934,1.0,"MULTILINESTRING ((261274.037 707315.032, 26127...",partially surveyed,False
2,osgb4000000004292780,Unknown,NaN,Single Carriageway,0.614588,false,S12000030,Stirling,30,NaN,...,NaN,NaN,5.0,4.1,614.587727,614.587727,1.0,"MULTILINESTRING ((255564.000 716216.000, 25558...",partially surveyed,False
3,osgb4000000004293117,Classified Unnumbered,NaN,Single Carriageway,1.174929,false,S12000030,Stirling,60,NaN,...,NaN,NaN,4.4,3.1,1174.929448,1174.929448,1.0,"MULTILINESTRING ((258317.000 732240.000, 25829...",partially surveyed,False
4,osgb4000000004293123,Classified Unnumbered,NaN,Single Carriageway,1.204896,false,S12000030,Stirling,60,NaN,...,NaN,NaN,5.8,4.0,1204.895873,1204.895873,1.0,"MULTILINESTRING ((259323.217 733055.412, 25933...",partially surveyed,False


In [156]:
def classify_speed(speed):
    if speed < 20:
        return "<20"
    elif speed < 30:
        return "20-30"
    elif speed < 40:
        return "30-40"
    else:
        return ">40"

def classify_aadt(aadt):
    if aadt < 500:
        return "<500"
    elif aadt < 1000:
        return "500-1000"
    elif aadt < 2000:
        return "1000-2000"
    elif aadt < 4000:
        return "2000-4000"
    elif aadt < 10000:
        return "4000-10000"
    else:
        return ">10000"


In [157]:
matrix_outwith = {
    "<500": {"<20": "Definitely Quietway", "20-30": "Definitely Quietway", "30-40": "Likely Quietway", ">40": "Make Quietway"},
    "500-1000": {"<20": "Definitely Quietway", "20-30": "Likely Quietway", "30-40": "Likely Quietway", ">40": "Make Quietway"},
    "1000-2000": {"<20": "Make Quietway", "20-30": "Make Quietway", "30-40": "Make Quietway", ">40": "Make Quietway"},
    "2000-4000": {"<20": "Make Quietway", "20-30": "Make Quietway", "30-40": "Make Quietway", ">40": "Make Traffic Free"},
    "4000-10000": {"<20": "Make Traffic Free", "20-30": "Make Traffic Free", "30-40": "Make Traffic Free", ">40": "Make Traffic Free"},
    ">10000": {"<20": "Make Traffic Free", "20-30": "Make Traffic Free", "30-40": "Make Traffic Free", ">40": "Make Traffic Free"}
}

matrix_within = {
    "<500": {"<20": "Definitely Quietway", "20-30": "Make Quietway", "30-40": "Make Quietway", ">40": "Make Traffic Free"},
    "500-1000": {"<20": "Definitely Quietway", "20-30": "Make Quietway", "30-40": "Make Quietway", ">40": "Make Traffic Free"},
    "1000-2000": {"<20": "Likely Quietway", "20-30": "Make Quietway", "30-40": "Make Quietway", ">40": "Make Traffic Free"},
    "2000-4000": {"<20": "Make Quietway", "20-30": "Make Quietway", "30-40": "Make Traffic Free", ">40": "Make Traffic Free"},
    "4000-10000": {"<20": "Make Quietway", "20-30": "Make Traffic Free", "30-40": "Make Traffic Free", ">40": "Make Traffic Free"},
    ">10000": {"<20": "Make Traffic Free", "20-30": "Make Traffic Free", "30-40": "Make Traffic Free", ">40": "Make Traffic Free"}
}


In [158]:
def assign_quietway(row):
    try:
        aadt_cat = classify_aadt(row['Modelled AADT'])
        speed_cat = classify_speed(row['85th %ile Speed - All Day'])

        if pd.isna(aadt_cat) or pd.isna(speed_cat):
            return None  # if either value is missing

        if row['within_settlement']:
            return matrix_within.get(aadt_cat, {}).get(speed_cat, None)
        else:
            return matrix_outwith.get(aadt_cat, {}).get(speed_cat, None)
    except:
        return None



In [159]:
quietway_assessment_intersect['propensity_for_quietway'] = quietway_assessment_intersect.apply(assign_quietway, axis=1)


In [160]:

quietway_assessment_intersect['propensity_for_quietway'].value_counts(dropna=False)


propensity_for_quietway
Make Quietway          4282
Definitely Quietway    2151
Make Traffic Free      1953
Likely Quietway        1421
Name: count, dtype: int64

In [161]:
quietway_assessment_intersect.head()

,OS Link ID,Road Classification,Road Classification Number,Form of Way,Length (km),Trunk Road,Highway Authority Code,Highway Authority Name,Speed Limit (mph),Speed Limit Evidence date,...,Number over 15 Above,roadwidth_average,roadwidth_minimum,length_total,length_in_buffer,percent_in_buffer,geometry,survey_flag,within_settlement,propensity_for_quietway
0,osgb4000000004292146,Classified Unnumbered,NaN,Single Carriageway,2.209182,false,S12000030,Stirling,60,NaN,...,NaN,5.2,NaN,2209.182012,2209.182012,1.0,"MULTILINESTRING ((264275.866 734983.274, 26433...",partially surveyed,False,Likely Quietway
1,osgb4000000004292724,Classified Unnumbered,NaN,Single Carriageway,0.656623,false,S12000030,Stirling,60,NaN,...,NaN,4.0,3.2,656.622934,656.622934,1.0,"MULTILINESTRING ((261274.037 707315.032, 26127...",partially surveyed,False,Definitely Quietway
2,osgb4000000004292780,Unknown,NaN,Single Carriageway,0.614588,false,S12000030,Stirling,30,NaN,...,NaN,5.0,4.1,614.587727,614.587727,1.0,"MULTILINESTRING ((255564.000 716216.000, 25558...",partially surveyed,False,Definitely Quietway
3,osgb4000000004293117,Classified Unnumbered,NaN,Single Carriageway,1.174929,false,S12000030,Stirling,60,NaN,...,NaN,4.4,3.1,1174.929448,1174.929448,1.0,"MULTILINESTRING ((258317.000 732240.000, 25829...",partially surveyed,False,Likely Quietway
4,osgb4000000004293123,Classified Unnumbered,NaN,Single Carriageway,1.204896,false,S12000030,Stirling,60,NaN,...,NaN,5.8,4.0,1204.895873,1204.895873,1.0,"MULTILINESTRING ((259323.217 733055.412, 25933...",partially surveyed,False,Definitely Quietway


In [162]:
print(quietway_assessment_intersect['Modelled AADT'].isnull().sum())

0


In [163]:
# Manually check the accuracy 
fields = [
    'within_settlement',
    'propensity_for_quietway',
    'Modelled AADT',
    '85th %ile Speed - All Day',
    'survey_flag'
]

quietway_assessment_intersect[fields].head(20)


,within_settlement,propensity_for_quietway,Modelled AADT,85th %ile Speed - All Day,survey_flag
0,False,Likely Quietway,136.175814,30.843303,partially surveyed
1,False,Definitely Quietway,114.560606,27.574892,partially surveyed
2,False,Definitely Quietway,44.311178,19.874551,partially surveyed
3,False,Likely Quietway,195.617638,31.306224,partially surveyed
4,False,Definitely Quietway,176.163950,29.754350,partially surveyed
5,False,Definitely Quietway,164.275586,25.477764,partially surveyed
6,False,Likely Quietway,126.448970,30.128726,partially surveyed
7,False,Definitely Quietway,48.634219,21.249335,fully surveyed
8,False,Definitely Quietway,83.218553,21.833424,fully surveyed
9,False,Definitely Quietway,384.750713,29.207544,partially surveyed


**After manual check, nothing is wrong from the table and the allocated quientness value. I planed to go back to the map to check the differences and see the exact value difference.**

In [ ]:
quietway_assessment_cici = quietway_assessment.loc[quietway_assessment_intersect.index]


In [166]:
quietway_assessment_cici= quietway_assessment.merge(
    quietway_assessment_intersect[['propensity_for_quietway','within_settlement']],
    left_index=True, right_index=True, how='left'
)

In [167]:
len(quietway_assessment_intersect)

9807

In [168]:
len(agilysis_selected_clean)

10478

In [169]:
len(quietway_assessment)

10478

In [170]:
len(quietway_assessment_cici)

10478

In [171]:
quietway_assessment_cici.head()

,OS Link ID,Road Classification,Road Classification Number,Form of Way,Length (km),Trunk Road,Highway Authority Code,Highway Authority Name,Speed Limit (mph),Speed Limit Evidence date,...,Number over 15 Above,roadwidth_average,roadwidth_minimum,length_total,length_in_buffer,percent_in_buffer,geometry,survey_flag,propensity_for_quietway,within_settlement
0,osgb4000000004292146,Classified Unnumbered,NaN,Single Carriageway,2.209182,false,S12000030,Stirling,60,NaN,...,NaN,5.2,NaN,2209.182012,2209.182012,1.0,"MULTILINESTRING ((264275.866 734983.274, 26433...",partially surveyed,Likely Quietway,False
1,osgb4000000004292724,Classified Unnumbered,NaN,Single Carriageway,0.656623,false,S12000030,Stirling,60,NaN,...,NaN,4.0,3.2,656.622934,656.622934,1.0,"MULTILINESTRING ((261274.037 707315.032, 26127...",partially surveyed,Definitely Quietway,False
2,osgb4000000004292780,Unknown,NaN,Single Carriageway,0.614588,false,S12000030,Stirling,30,NaN,...,NaN,5.0,4.1,614.587727,614.587727,1.0,"MULTILINESTRING ((255564.000 716216.000, 25558...",partially surveyed,Definitely Quietway,False
3,osgb4000000004293117,Classified Unnumbered,NaN,Single Carriageway,1.174929,false,S12000030,Stirling,60,NaN,...,NaN,4.4,3.1,1174.929448,1174.929448,1.0,"MULTILINESTRING ((258317.000 732240.000, 25829...",partially surveyed,Likely Quietway,False
4,osgb4000000004293123,Classified Unnumbered,NaN,Single Carriageway,1.204896,false,S12000030,Stirling,60,NaN,...,NaN,5.8,4.0,1204.895873,1204.895873,1.0,"MULTILINESTRING ((259323.217 733055.412, 25933...",partially surveyed,Definitely Quietway,False


In [174]:
quietway_assessment_cici.to_file("quietway_assessment_cici.shp", driver="ESRI Shapefile")


/var/folders/74/ch5llxvd3klg6n6_l6_s335w0000gn/T/ipykernel_70864/449450616.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  quietway_assessment_cici.to_file("quietway_assessment_cici.shp", driver="ESRI Shapefile")


# Statistical summary

In [172]:
quietway_assessment_cici['propensity_for_quietway'].value_counts(dropna=False)


propensity_for_quietway
Make Quietway          4282
Definitely Quietway    2151
Make Traffic Free      1953
Likely Quietway        1421
NaN                     671
Name: count, dtype: int64

In [173]:
quietway_assessment_intersect['propensity_for_quietway'].value_counts(dropna=False)


propensity_for_quietway
Make Quietway          4282
Definitely Quietway    2151
Make Traffic Free      1953
Likely Quietway        1421
Name: count, dtype: int64

In [176]:
proportions = quietway_assessment_cici['propensity_for_quietway'].value_counts(normalize=True, dropna=False)


In [177]:
proportions

propensity_for_quietway
Make Quietway          0.408666
Definitely Quietway    0.205287
Make Traffic Free      0.186391
Likely Quietway        0.135617
NaN                    0.064039
Name: proportion, dtype: float64

In [178]:
# Make a clean copy
df = quietway_assessment_cici.copy()

# Calculate lengths
df['length_m'] = df.geometry.length
df['length_km'] = df['length_m'] / 1000
df['length_miles'] = df['length_m'] * 0.000621371

df['quietway_class'] = df['propensity_for_quietway'].fillna('no volume and/or 85th percentile speed').str.lower().str.strip()

# Group and summarise
summary = df.groupby('quietway_class').agg(
    count=('geometry', 'count'),
    length_m=('length_m', 'sum'),
    length_km=('length_km', 'sum'),
    length_miles=('length_miles', 'sum')
).reset_index()

summary[['length_m', 'length_km', 'length_miles']] = summary[['length_m', 'length_km', 'length_miles']].round(0)

# Desired order of categories
desired_order = [
    'definitely quietway',
    'likely quietway',
    'make quietway',
    'make traffic free',
    'no volume and/or 85th percentile speed'
]

# Order and add total
summary['quietway_class'] = pd.Categorical(summary['quietway_class'], categories=desired_order, ordered=True)
summary = summary.sort_values('quietway_class')

grand_total = pd.DataFrame({
    'quietway_class': ['Total'],
    'count': [summary['count'].sum()],
    'length_m': [summary['length_m'].sum()],
    'length_km': [summary['length_km'].sum()],
    'length_miles': [summary['length_miles'].sum()]
})

summary = pd.concat([summary, grand_total], ignore_index=True)

In [179]:
summary

,quietway_class,count,length_m,length_km,length_miles
0,definitely quietway,2151,275905.0,276.0,171.0
1,likely quietway,1421,403484.0,403.0,251.0
2,make quietway,4282,481289.0,481.0,299.0
3,make traffic free,1953,121731.0,122.0,76.0
4,no volume and/or 85th percentile speed,671,88695.0,89.0,55.0
5,Total,10478,1371104.0,1371.0,852.0
